<a href="https://colab.research.google.com/github/jeffblackadar/crane-pompeii/blob/master/evernote_extract_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is a proof of concept to extract images from a notebook in Evernote.
* It lists all of the notebooks
* For a selected notebook, it loops through all notes in it.
* For each note, it saves images known as "resources" as a file.

## Tokens
A token is needed: https://sandbox.evernote.com/api/DeveloperToken.action

## Credit to:
### Evernote API & Python 3.x, https://xbopp.com/evernote-to-python1/
### Evernote Developer Documentation https://dev.evernote.com/doc/
### Evernote API reference https://dev.evernote.com/doc/reference/
### Search https://dev.evernote.com/doc/articles/search.php
### Resources https://dev.evernote.com/doc/articles/resources.php

In [1]:
!pip install git+https://github.com/evernote/evernote-sdk-python3.git
!pip install oauth2

  Cloning https://github.com/evernote/evernote-sdk-python3.git to /tmp/pip-req-build-kya5otyw
  Running command git clone -q https://github.com/evernote/evernote-sdk-python3.git /tmp/pip-req-build-kya5otyw
  Created wheel for evernote3: filename=evernote3-1.25.0-cp36-none-any.whl size=143704 sha256=a23621cc86d54a050cdb667ddb6018f766f0da40392fede406aaaf5295e8ab18
  Stored in directory: /tmp/pip-ephem-wheel-cache-vjy7oplc/wheels/63/4a/27/69fc0e9c3afe2951dcfc496e595173fe71a3d456f31c8d0666
Successfully built evernote3


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import json
#import hashlib
#import binascii
import evernote.edam.userstore.constants as UserStoreConstants
#import evernote.edam.type.ttypes as Types
import os
from evernote.api.client import EvernoteClient
from evernote.edam.notestore.ttypes import NoteFilter, NotesMetadataResultSpec
from PIL import Image

import io
import binascii

# https://xbopp.com/evernote-to-python1/
# Real applications authenticate with Evernote using OAuth, but for the
# purpose of exploring the API, you can get a developer token that allows
# you to access your own Evernote account. To get a developer token, visit
# https://sandbox.evernote.com/api/DeveloperToken.action

# The json file stores the EVERNOTE_DEV_TOKEN so we don't have it in the code.
# The json file looks like this:
# {"EVERNOTE_DEV_TOKEN": "S=..... etc...."}
# cv.json contains the credentials for this program.

settings_path = "/content/drive/MyDrive/william_white/"
evernote_settings_file = settings_path+'evernote.json'
with open(evernote_settings_file, "r") as read_file:
    evernote_auth_data = json.load(read_file)

auth_token = evernote_auth_data["EVERNOTE_DEV_TOKEN"]

sandbox=True
china=False

#client = EvernoteClient(token=auth_token, sandbox=sandbox,china=china)
client = EvernoteClient(token=auth_token, sandbox=sandbox)

user_store = client.get_user_store()
version_ok = user_store.checkVersion(
    "Evernote EDAMTest (Python)",
    UserStoreConstants.EDAM_VERSION_MAJOR,
    UserStoreConstants.EDAM_VERSION_MINOR
)
print("Is my Evernote API version up to date? ", str(version_ok))
if not version_ok:
    exit(1)

note_store = client.get_note_store()

# List all of the notebooks in the user's account
notebooks = note_store.listNotebooks()
print("Found ", len(notebooks), " notebooks:")
myNotebookGuid = ""
for notebook in notebooks:
    print("notebook title: ",notebook.name, "guid: ", notebook.guid)
    # look for a specific notebook
    if(notebook.name == "First Notebook"):
        myNotebookGuid = notebook.guid    

if myNotebookGuid == "":
    print("Specific notebook note found. Is the name spelled right?")
    exit(1)

#sort of pointless - since we already have a handle on it above.
myNotebook = note_store.getNotebook(myNotebookGuid)


# filter to get notes inside the notebook
filter = NoteFilter()
filter.notebookGuid = myNotebookGuid
filter.ascending = False

spec = NotesMetadataResultSpec()
spec.includeTitle = True

ourNoteList = note_store.findNotesMetadata(auth_token, filter, 0, 100, spec)
#for note in ourNoteList.notes:
    #print(note.guid)
    # https://dev.evernote.com/doc/reference/NoteStore.html#Fn_NoteStore_getNoteContent
    #content = note_store.getNoteContent(note.guid)
    #print(content)

image_destination = "/content/drive/MyDrive/crane_amuqall_old/"

for note in ourNoteList.notes:
    wholeNote = note_store.getNote(auth_token, note.guid, True, True, True, True) 
    print ("Content length: %d" % wholeNote.contentLength)
    note_resources = wholeNote.resources
    for note_resource in note_resources:
        file_content = note_resource.data.body
        file_type = note_resource.mime
        file_name = note_resource.attributes.fileName
        # Thanks to https://stackoverflow.com/questions/42339876/error-unicodedecodeerror-utf-8-codec-cant-decode-byte-0xff-in-position-0-in
        decode_bytes = file_content.decode('ISO-8859-1')
        resource_bytes = bytes(decode_bytes, 'ISO-8859-1')
        stream = io.BytesIO(resource_bytes)
        img = Image.open(stream)
        print("saving",file_name)
        img.save(os.path.join(image_destination,file_name))

Is my Evernote API version up to date?  True
Found  1  notebooks:
notebook title:  First Notebook guid:  dee129fb-5d4a-40c5-b4f3-4232f26165fc
Content length: 335
saving IMG-20120713-00160.jpg
saving IMG-20120713-00171.jpg
Content length: 261
saving IMG-20120713-00186.jpg
Content length: 268
saving BlueMtHearthforShawnJeff.jpg
